In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../..")

import os
import tempfile
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from models.basic import CNN, BidirectionalRNN
from torch import nn
from models.stroke_recovery_loss import StrokeLoss
import torch
from models.CoordConv import CoordConv
from trainers import TrainerStrokeRecovery
from hwr_utils.stroke_dataset import StrokeRecoveryDataset, read_img
from hwr_utils.stroke_recovery import *
from hwr_utils import utils
from torch.optim import lr_scheduler
from timeit import default_timer as timer
from train_stroke_recovery import graph

folder = Path("online_coordinate_data/3_stroke_vSmall")
folder = Path("online_coordinate_data/8_stroke_vSmall_16")
folder = Path("online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull")

print(os.getcwd())
x_relative_positions = True
test_size = 5
train_size = None
batch_size=32

test_dataset=StrokeRecoveryDataset([folder / "test_online_coords.json"],
                        img_height = 60,
                        num_of_channels = 1.,
                        max_images_to_load = test_size,
                        root=r"../../data",
                        x_relative_positions=x_relative_positions
                        )

test_dataloader = DataLoader(test_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=3,
                              collate_fn=test_dataset.collate,
                              pin_memory=False)

device="cuda"
example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
output = Path("./TEST_GRAPHING")

# This graphs the directions etc.
# graph(example, save_folder=output, x_relative_positions=x_relative_positions)

fd, path = tempfile.mkstemp()
try:
    draw_strokes(normalize_stroke_list(example.raw), ratio, save_path=path, line_width=.8)
    img = read_img(path)
finally:
    os.remove(path)

print(img)


/media/data/GitHub/simple_hwr/hwr_utils/dev
../../data/online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull/test_online_coords.json


INFO:root.hwr_utils.stroke_dataset:('Original dataloader size', 501)
INFO:root.hwr_utils.stroke_dataset:('Dataloader size', 5)
100%|██████████| 5/5 [00:00<00:00, 866.34it/s]


AttributeError: 'dict' object has no attribute 'raw'

In [4]:
print(example.keys())

dict_keys(['line_imgs', 'gt', 'gt_list', 'x_relative', 'label_lengths', 'paths', 'x_func', 'y_func', 'start_times'])


In [10]:
print(example["gt"].shape)

torch.Size([5, 869, 4])


In [ ]:
np.r_(start_indices, end_indices)

In [ ]:
array = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
start_indices = np.where(array)[0]
start_point_ct = len(start_indices)
total_point_ct = len(array)
end_indices = np.append((start_indices-1)[1:], total_point_ct)

xs = np.array(range(0, total_point_ct))*1.5
x = np.interp(xs, np.append(start_indices*1.0, end_indices), np.append(np.ones(start_point_ct), np.zeros(start_point_ct)))

print(x, xs, np.append(start_indices*1.0, end_indices))

In [ ]:
array = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
start_indices = np.where(array)[0]
start_point_ct = len(start_indices)
total_point_ct = len(array)
xs = np.repeat(start_indices,2)
xs[::2] -= 1
xs = np.append(xs[1:], total_point_ct)
ys = np.zeros(2*start_point_ct)
ys[1::2] += 1
print(xs, ys)

# make it distance traveled!

interp_xs = np.array(range(0, total_point_ct))
out = np.interp(interp_xs, xs, ys)
print(out)

In [ ]:
x = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
start_indices = np.where(x)
#end_indices = start_indices-1

print(start_indices)

In [ ]:
m = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
np.append(m, 599, 5)

TypeError: 'RClass' object is not callable

In [61]:
array = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
start_indices = np.where(array)[0]
start_point_ct = len(start_indices)
total_point_ct = len(array)
end_indices = np.append((start_indices-1)[1:], total_point_ct)

xs = np.array(range(0, total_point_ct))*1.5
x = np.interp(xs, np.append(start_indices*1.0, end_indices), np.append(np.ones(start_point_ct), np.zeros(start_point_ct)))

print(x, xs, np.append(start_indices*1.0, end_indices))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] [ 0.   1.5  3.   4.5  6.   7.5  9.  10.5 12.  13.5 15.  16.5 18.  19.5
 21.  22.5 24. ] [ 0.  8. 14.  7. 13. 17.]


In [79]:
array = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
start_indices = np.where(array)[0]
start_point_ct = len(start_indices)
total_point_ct = len(array)
xs = np.repeat(start_indices,2)
xs[::2] -= 1
xs = np.append(xs[1:], total_point_ct)
ys = np.zeros(2*start_point_ct)
ys[1::2] += 1
print(xs, ys)

# make it distance traveled!

interp_xs = np.array(range(0, total_point_ct))
out = np.interp(interp_xs, xs, ys)
print(out)

[ 0  7  8 13 14 17] [0. 1. 0. 1. 0. 1.]
[0.         0.14285714 0.28571429 0.42857143 0.57142857 0.71428571
 0.85714286 1.         0.         0.2        0.4        0.6
 0.8        1.         0.         0.33333333 0.66666667]


In [ ]:
x = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
start_indices = np.where(x)
#end_indices = start_indices-1

print(start_indices)

In [36]:
m = np.array([1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
np.append(m, 599, 5)

IndexError: index 599 is out of bounds for axis 0 with size 17